In [1]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.models import vit_b_16  # Import a pre-trained ViT model
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary as model_summary
from tqdm.auto import tqdm

# Definir transformaciones para los datos de entrenamiento y validación
transform = transforms.Compose([
  transforms.Resize((224, 224)),  # Redimensionar las imágenes a 224x224 píxeles
  transforms.ToTensor(),          # Convertir las imágenes a tensores
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [2]:

# Definir el tamaño del batch
batch_size = 32

# Crear los conjuntos de datos
train_dataset = datasets.CIFAR10(root='/content/drive/MyDrive/iamgenes/jet2000q', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='/content/drive/MyDrive/iamgenes/jet2000q'', train=False, download=True, transform=transform)

# Split train dataset into train and validation sets 80% and 20%
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print(f"Train Data: {len(train_dataset)}")
print(f"Validation Data: {len(val_dataset)}")
print(f"Test Data: {len(test_dataset)}")

# Crear los DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



100%|██████████| 170498071/170498071 [00:29<00:00, 5832245.98it/s] 


Extracting /Users/juvenalbassa/Desktop/Data5types2000/Jets5type2000/cifar-10-python.tar.gz to /Users/juvenalbassa/Desktop/Data5types2000/Jets5type2000


100%|██████████| 170498071/170498071 [00:42<00:00, 4042099.25it/s]


Extracting /Users/juvenalbassa/Desktop/Data5types2000/Jets5types-500/cifar-10-python.tar.gz to /Users/juvenalbassa/Desktop/Data5types2000/Jets5types-500
Train Data: 40000
Validation Data: 10000
Test Data: 10000


In [3]:

# Definir hiperparámetros
num_epochs = 5
lr = 3e-5  # Learning rate
gamma = 0.7 # Factor de reducción de la tasa de aprendizaje

# Cargar el modelo preentrenado
model = vit_b_16(pretrained=True)
model.heads.head = nn.Linear(model.heads.head.in_features, 10)  # Ajustar para CIFAR-10 (10 clases)



/Applications/anaconda3/envs/scikit-hep/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Applications/anaconda3/envs/scikit-hep/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Definir dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model.to(device)
if device == 'cuda':
   print(torch.cuda.get_device_name(0))

# Definir función de pérdida, optimizador y scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

Device: cpu


In [5]:
# Definir función de entrenamiento
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_loss = float('inf')
    model_file_name = 'best_model.pt'
    save_path ='/content/drive/MyDrive/modelos' # you have to create the file and put the  pathname

    # Crear el directorio si no existe
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        scheduler.step()
        
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")
        
        # Guardar el modelo si es el mejor hasta ahora
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), os.path.join(save_path, model_file_name))
            print('New best model saved!')

    print("Entrenamiento completado")

# Entrenar el modelo
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/5, Train Loss: 0.17393059995919466, Validation Loss: 0.10012506270192016
New best model saved!
Epoch 2/5, Train Loss: 0.03971489501502365, Validation Loss: 0.10654931121874756
Epoch 3/5, Train Loss: 0.01573250534129329, Validation Loss: 0.08588722579797861
New best model saved!
Epoch 4/5, Train Loss: 0.005504002051730641, Validation Loss: 0.09671756036208791
Epoch 5/5, Train Loss: 0.0026516811225563287, Validation Loss: 0.08928995165053737
Entrenamiento completado


In [6]:

def evaluate_model(model, test_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test images: {accuracy}%')

In [7]:
# Load the best model
model_file_name = 'best_model_2402422_v01.pt'
best_model_state_dict = torch.load(model_file_name)
model.load_state_dict(best_model_state_dict)

<All keys matched successfully>

In [8]:
# Evaluate the model
evaluate_model(model, test_loader)

Accuracy on test images: 96.77%
